In [1]:
#!pip install -q sentence-transformers langchain langchain-community langchain-chroma chromadb

In [2]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

/Users/alemanrique/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-05-13 22:04:19.203365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd

#QA
qa_data = [
    {
        "question": "What is tuition cost for the program?",
        "answer": "Tuition for the MS in Applied Data Science program: $5,967 per course/$71,604 total tuition"
    },
    {
        "question": "What scholarships are available for the program?",
        "answer": "The Data Science Institute Scholarship, MS in Applied Data Science Alumni Scholarship"
    },
    {
        "question": "What are the minimum scores for the TOEFL and IELTS English Language Requirement?",
        "answer": "Minimum scores for the Master’s in Applied Data Science program: TOEFL, 102 (no subscore requirement); IELTS, 7 (no subscore requirement)."
    },
    {
        "question": "Is there an application fee waiver?",
        "answer": "For questions regarding an application fee waiver, please refer to the Physical Sciences Division fee waiver policy."
    },
    {
        "question": "What are the deadlines for the in-person program?",
        "answer": """November 7, 2024 - Priority Application Deadline
December 4, 2024 - Scholarship Priority Deadline
January 21, 2025 - International Application Deadline (requiring visa sponsorship from UChicago)
March 4, 2025 - Second Priority Application Deadline
May 6, 2025 - Third Priority Application Deadline
June 23, 2025 - Final Application Deadline"""
    },
    {
        "question": "How long will it take for me to receive a decision on my application?",
        "answer": "In-Person application decisions are released approximately 1 to 2 months after each respected deadline. Online application decisions are released on a rolling basis"
    },
    {
        "question": "Can I set up an advising appointment with the enrollment management team?",
        "answer": "Yes, meet your admissions counselor by scheduling an appointment https://apply-psd.uchicago.edu/portal/applied-data-science"
    },
    {
        "question": "Where can I mail my official transcripts?",
        "answer": "The University of Chicago\nAttention: MS in Applied Data Science Admissions\n455 N Cityfront Plaza Dr., Suite 950\nChicago, Illinois 60611"
    },
    {
        "question": "Does the Master’s in Applied Data Science Online program provide visa sponsorship?",
        "answer": "Only our In-Person, Full-Time program is Visa eligible"
    },
    {
        "question": "How do I apply to the MBA/MS program?",
        "answer": "Applicants interested in the Joint MBA/MS degree will apply through Booth’s centralized, joint-application process. Applicants should complete the Chicago Booth Full-Time MBA application and select the MBA/MS in Applied Data Science as their program of interest"
    },
    {
        "question": "Is the MS in Applied Data Science program STEM/OPT eligible?",
        "answer": "The MS in Applied Data Science program is STEM/OPT eligible"
    },
    {
        "question": "How many courses must you complete to earn UChicago’s Master’s in Applied Data Science?",
        "answer": "To earn the MS-ADS degree students must successfully complete 12 courses (6 core, 4 elective, 2 Capstone) and our tailored Career Seminar"
    }, 
    {
        "question": "What programs options are available in the MS Applied Data Science program?",
        "answer": "In-person program, Online progran and Joint MBA/MS program"
    }, 
    {
        "question": "How many letters of recommendation do you need for the program?",
        "answer": "The MS in Applied Data Science program requires two letters of recommendation"
    },
    {
        "question": "Who should write my letters of recommendation?",
        "answer": "We strongly recommend that at least one letter is written by someone such as a direct manager/supervisor or internship supervisor who can attest to skills you demonstrated or gained though a professional workplace experience (e.g., leadership, teamwork, collaboration, initiative, management, other)."
    },
    {
        "question": "Does the MS in Applied Data Science provide scholarships?",
        "answer": "The MS in Applied Data Science program offers partial tuition scholarships to top applicants. These scholarships do not require a separate application but it is recommended that candidates submit their applications ahead of the early deadline to maximize their chances of securing a scholarship."
    },
    {
        "question": "Where is the campus located?",
        "answer": "Our campus is strategically situated at the NBC Tower and Gleacher Center in downtown Chicago’s Streeterville neighborhood"
    },
    {
        "question": "Where is the Data Science Institute located?",
        "answer": "Located within Hyde Park campus"
    },
    {
        "question": "How many classess should I take per quarter?",
        "answer": "Full-time students take 3 classes per quarter (300 units). Part-time students take 2 classes per quarter (200 units)"
    }, 
    {
        "question": "When are classes held for part-time students?",
        "answer": "Most courses are from 6-9pm Monday through Thursday with some offered on Fridays and Saturdays. This allows you to work in an internship and/or job during the program"
    }, 
    {
        "question": "What are the four foundational courses?",
        "answer": "Introduction to Statistical Concepts (Foundational, optional), R for Data Science (Foundational, optional), Python for Data Science (Foundational, optional), Advanced Linear Algebra for Machine Learning (Foundational, optional)"
    },
    {
        "question": "Is there an application fee?",
        "answer": "There is a $90 non-refundable application fee"
    },
    {
        "question": "What is the purpose of the capstone projects in the MS in Applied Data Science program?",
        "answer": "Capstone projects allow students to address real-world challenges across industries, showcasing practical and impactful data science solutions. Outstanding projects are recognized with awards such as 'Best in Show'."
    },
    {
        "question": "What does the MBA + Applied Data Science dual degree offer?",
        "answer": "The dual degree equips students to lead in data-centric environments by bridging technical and managerial domains, making them effective decision-makers who understand both business and data science."
    },
    {
        "question": "What is the structure of the capstone experience in the MS in Applied Data Science program?",
        "answer": "The capstone is a project-based course where students work in teams to solve real-world problems for clients from industry, academia, and social impact organizations, applying data science in a practical setting."
    },
    {
        "question": "What kind of interdisciplinary skills are emphasized in the MS in Applied Data Science curriculum?",
        "answer": "The curriculum emphasizes a combination of computational and analytical skills, domain knowledge, communication, and ethics, preparing students for broad application of data science in various sectors."
    },
    {
        "question": "What types of real-world experiences do students gain during the MS in Applied Data Science program?",
        "answer": "Students gain hands-on experience through capstone projects, industry collaborations, and events such as company site visits, helping them build practical skills that are directly applicable to the workforce."
    },
    {
        "question": "How does the MS in Applied Data Science program prepare students for ethical challenges in data science?",
        "answer": "Ethics is integrated into the curriculum alongside technical skills to ensure graduates are equipped to consider the social impact of data science solutions and apply responsible practices in real-world contexts."
    }

]

# Convert to DataFrame
qa_df = pd.DataFrame(qa_data)

In [4]:
#save qa_df as csv
qa_df.to_csv("qa_data.csv", index=False)

In [5]:
qa_df

,question,answer
0,What is tuition cost for the program?,Tuition for the MS in Applied Data Science pro...
1,What scholarships are available for the program?,"The Data Science Institute Scholarship, MS in ..."
2,What are the minimum scores for the TOEFL and ...,Minimum scores for the Master’s in Applied Dat...
3,Is there an application fee waiver?,For questions regarding an application fee wai...
4,What are the deadlines for the in-person program?,"November 7, 2024 - Priority Application Deadli..."
5,How long will it take for me to receive a deci...,In-Person application decisions are released a...
6,Can I set up an advising appointment with the ...,"Yes, meet your admissions counselor by schedul..."
7,Where can I mail my official transcripts?,The University of Chicago\nAttention: MS in Ap...
8,Does the Master’s in Applied Data Science Onli...,"Only our In-Person, Full-Time program is Visa ..."
9,How do I apply to the MBA/MS program?,Applicants interested in the Joint MBA/MS degr...


In [6]:
train_samples = [InputExample(texts=[row['question'], row['answer']], label=1.0) for _, row in qa_df.iterrows()]
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=4)

In [7]:
model = SentenceTransformer("all-MiniLM-L6-v2")
train_loss = losses.MultipleNegativesRankingLoss(model)

In [8]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator(
    sentences1=qa_df["question"].tolist(),
    sentences2=qa_df["answer"].tolist(),
    scores=[1.0] * len(qa_df)
)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=4,
    warmup_steps=10,
    show_progress_bar=True,
    evaluation_steps=5,
    output_path="./app_model"
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Pearson Cosine,Spearman Cosine
5,No log,No log,nan,nan
7,No log,No log,nan,nan
10,No log,No log,nan,nan
14,No log,No log,nan,nan
15,No log,No log,nan,nan
20,No log,No log,nan,nan
21,No log,No log,nan,nan
25,No log,No log,nan,nan
28,No log,No log,nan,nan


/Users/alemanrique/anaconda3/lib/python3.11/site-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:203: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
/Users/alemanrique/anaconda3/lib/python3.11/site-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:204: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_spearman, _ = spearmanr(labels, scores)
/Users/alemanrique/anaconda3/lib/python3.11/site-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:203: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
/Users/alemanrique/anaconda3/lib/python3.11/site-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:204: ConstantInputWarning: An input array is constant; the correlation coe

In [9]:
model_save_path = "./app_model"
os.makedirs(model_save_path, exist_ok=True)
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

Model saved to: ./app_model


In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="./app_model")

/var/folders/2l/hvbq_9n91959jb6bcwjbyw7c0000gn/T/ipykernel_46226/3899561710.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="./app_model")


In [11]:
documents = [
    Document(page_content=row['answer'], metadata={"question": row['question']})
    for _, row in qa_df.iterrows()
]

In [13]:
from langchain_community.vectorstores import Chroma

persist_path = "chroma_v3"
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=persist_path
)
vectorstore.persist()
print(f"Chroma vectorstore persisted to: {persist_path}")

Chroma vectorstore persisted to: chroma_v3


/var/folders/2l/hvbq_9n91959jb6bcwjbyw7c0000gn/T/ipykernel_46226/3896723002.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [15]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

# Load your OpenAI key from .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Step 1: Load and split documents
loader = TextLoader("uchicago_datascience_cleaned.txt")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(documents)

#model 
os.environ["HF_HUB_OFFLINE"] = "1"

# Use correct local model path
model_path = os.path.abspath("app_model")  # No __file__

# Load model with renamed model.safetensors file
embedding_model = HuggingFaceEmbeddings(model_name=model_path)

# Step 3: Create or load vector store
persist_dir = "chroma_v3"
db = Chroma(persist_directory=persist_dir, embedding_function=embedding_model)
retriever = db.as_retriever(search_kwargs={"k": 3})

# Step 4: Generator (GPT-3.5 for now)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

# Step 5: Build RAG chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# Step 6: Ask a test question
query = "What is the structure of the capstone experience?"
result = qa_chain({"query": query})

# Print the result and sources
print("📌 Answer:\n", result["result"])
print("\n📚 Source Documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n--- Source {i+1} ---\n", doc.page_content[:500])


/var/folders/2l/hvbq_9n91959jb6bcwjbyw7c0000gn/T/ipykernel_46226/2472980180.py:43: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


📌 Answer:
 The capstone experience typically involves students working in teams to solve real-world problems for clients from various sectors, applying data science skills in a practical setting. The structure may include project planning, data collection and analysis, model building, and presentation of findings and recommendations to the client. Additionally, there may be milestones, checkpoints, and feedback sessions throughout the project duration to ensure progress and quality of work.

📚 Source Documents:

--- Source 1 ---
 The capstone is a project-based course where students work in teams to solve real-world problems for clients from industry, academia, and social impact organizations, applying data science in a practical setting.

--- Source 2 ---
 The capstone is a project-based course where students work in teams to solve real-world problems for clients from industry, academia, and social impact organizations, applying data science in a practical setting.

--- Source 3 ---
 

In [ ]:
# Step 6: Ask a test question
query = "how much is tuition?"
result = qa_chain({"query": query})

# Print the result and sources
print("📌 Answer:\n", result["result"])

📌 Answer:
 The total tuition for the MS in Applied Data Science program is $71,604.
